In [159]:
from collections import defaultdict
import numpy as np
import re

In [160]:
txt_data = [
    "this is a very good thing to buy if you don't want your child to have a phone, but they want to be able to contact their friends, listen to music, or play games. They don't break by themselves, so if something goes wrong, there is a defect, it was handled incorrectly, or you/ your child did something to mess it up. I recommend a protective case and a screen protector. You can download about 10-15 apps, and a lot of songs. But it does have little storage, so make sure you put the photos on your computer. if you are using it for taking pictures and videos or wants a lot of apps or songs, i recommend 36 or 64 gb. enjoy!",
    "So far I'm extremely disappointed with this iPod Touch. I find it much more difficult to use. I do not find it user friendly. I feel that I should have simply purchased an iPhone instead of this product. I upgrade from the classic iPod which is still in my opinion amazing and I will probably continue to use. So far I am not happy with this iPod Touch and would not recommend it to anyone",
    "received this iPod for Christmas (2016)! It works great and takes perfect photos. I love the fact that I can facetime with any other Apple phone. I'm still getting familiar with all the things I can do with it. The only downside is that it did not come with a charging block. Luckily, we have plenty extras. So far it works great and I am very pleased. The gold display is perfect. So, I am going to give this iPod five stars. Maddy 11 years old.",
    "I love my iPod touch so much! I use it for basically everything-even as a mini iPhone. But I do think the screen is a little small, but for now it will work-I'm getting an iPad-but I simply love it! If you are looking for a great device for your littler children or teens or tweens or even an adult I highly reccomend-and I have a tip for you-for your littler children, you probably don't want them to be on the internet sometimes when you can't supervise, there's a way to take Safari on and off. I really recommend this iPod to everyone, especially children!",
    "I just bought this iPod touch today and it's my first real iPod. I have a friend who has had this iPod for a long time and he loves his too. It is very small which can be a pro to some and a con to others (for me it's a con but I'll get used to it). I love that I can clear up the space from my iPhone 6s Plus by moving all my music and games over to this iPod and using the phone just for texting and calling."
]

In [161]:
vocab = set()
doc_term = defaultdict(dict)
for txt in txt_data:
    terms = [re.sub(r'[^\w\s]','',s).lower() for s in txt.split(' ')]
    vocab = vocab.union(terms)
    
for word in vocab:
    for doc_id in range(len(txt_data)):
        if doc_id not in doc_term.keys():
            doc_term[doc_id] = defaultdict(int)
        doc_term[doc_id][word] = 0

for word in vocab:
    for doc_id, txt in enumerate(txt_data):
        terms = [re.sub(r'[^\w\s]','',s).lower() for s in txt.split(' ')]
        if word in terms:
            doc_term[doc_id][word] = 1

In [175]:
w1 = 'iphone'
N = len(doc_term.keys())

info_pairs = []
for w2 in vocab:
    p_X_w1_1 = 0
    p_X_w2_1 = 0
    p_X_w1_1_w2_1 = 0
    for doc_id in doc_term.keys():
        w1_present = False
        w2_present = False
        if doc_term[doc_id][w1] > 0:
            w1_present = True
            p_X_w1_1 += 1
        if doc_term[doc_id][w2] > 0:
            w2_present = True
            p_X_w2_1 += 1
        if w1_present and w2_present:
            p_X_w1_1_w2_1 += 1

    # P(X_w1 = 1)
    p_X_w1_1 = (p_X_w1_1 + 0.05) / (float(N) + 1)

    # P(X_w2 = 1)
    p_X_w2_1 = (p_X_w2_1 + 0.05) / (float(N) + 1)

    # P(X_w1=1, X_w2=1)
    p_X_w1_1_w2_1 = (p_X_w1_1_w2_1 + 0.025) / (float(N) + 1)

    # P(X_w1 = 0) = 1 - P(X_w1 = 1)
    p_X_w1_0 = 1. - p_X_w1_1

    # P(X_w2 = 0) = 1 - P(X_w2 = 1)
    p_X_w2_0 = 1. - p_X_w2_1

    # P(X_w1=1, X_w2=0) + P(X_w1=1, X_w2=1) = P(X_w1=1)
    p_X_w1_1_w2_0 = p_X_w1_1 - p_X_w1_1_w2_1

    # P(X_w1=0, X_w2=1) + P(X_w1=1, X_w2=1) = P(X_w2=1)
    p_X_w1_0_w2_1 = p_X_w2_1 - p_X_w1_1_w2_1

    # P(X_w1=0, X_w2=0) + P(X_w1=0, X_w2=1) = P(X_w1=0)
    p_X_w1_0_w2_0 = p_X_w1_0 - p_X_w1_0_w2_1
    
    I_w1_w2 = (p_X_w1_1_w2_1 * np.log2(p_X_w1_1_w2_1 / (p_X_w1_1 * p_X_w2_1))) +\
            (p_X_w1_0_w2_0 * np.log2(p_X_w1_0_w2_0 / (p_X_w1_0 * p_X_w2_0))) +\
            (p_X_w1_1_w2_0 * np.log2(p_X_w1_1_w2_0 / (p_X_w1_1 * p_X_w2_0))) +\
            (p_X_w1_0_w2_1 * np.log2(p_X_w1_0_w2_1 / (p_X_w1_0 * p_X_w2_1)))
    
    info_pairs.append((I_w1_w2, w2))

In [176]:
sorted(info_pairs, key=lambda x:x[0], reverse=True)[:10]

[(0.93027165346995799, 'touch'),
 (0.93027165346995799, 'iphone'),
 (0.93027165346995799, 'my'),
 (0.45039734091734002, 'did'),
 (0.45039734091734002, 'photos'),
 (0.4235643331761012, 'small'),
 (0.4235643331761012, 'simply'),
 (0.4235643331761012, 'use'),
 (0.4235643331761012, 'from'),
 (0.4235643331761012, 'much')]